In [1]:
import pandas as pd
##read sheets from the excel file individually and and then filter all rows out that arent the class
# Second step is necessary as we just hidden the other classes in the individual excel sheets 

dataset_file_name : str = 'custom_dataset.xlsx'

net_zero = pd.read_excel(io=dataset_file_name, sheet_name='Net Zero')
net_zero = net_zero[net_zero['end_target'] == "Net zero"]

carbon_neutrality = pd.read_excel(io=dataset_file_name, sheet_name='Carbon Neutrality')
carbon_neutrality = carbon_neutrality[carbon_neutrality['end_target'] == "Carbon neutral(ity)"]

emission_reduction_target = pd.read_excel(io=dataset_file_name, sheet_name='Emissions Reduction Target')
emission_reduction_target = emission_reduction_target[emission_reduction_target['end_target'] == "Emissions reduction target"]

no_target = pd.read_excel(io=dataset_file_name, sheet_name='No Target')
no_target = no_target[no_target['end_target'] == "No target"]

In [2]:
#Remove every row where the custom_text is NaN so only our rows remain
net_zero.dropna(inplace=True, subset=['custom_text'])
carbon_neutrality.dropna(inplace=True, subset=['custom_text'])
emission_reduction_target.dropna(inplace=True, subset=['custom_text'])
no_target.dropna(inplace=True, subset=['custom_text'])

In [3]:
net_zero['Source'] = 'Net Zero'
carbon_neutrality['Source'] = 'Carbon Neutrality'
emission_reduction_target['Source'] = 'Emissions Reduction Target'
no_target['Source'] = 'No Target'

#Concat into a single dataframe
all_targets = pd.concat([net_zero, carbon_neutrality, emission_reduction_target, no_target], ignore_index=True, axis=0)

all_targets['end_target_year'] = pd.to_numeric(all_targets['end_target_year'], errors='coerce')
all_targets['end_target_year'] = all_targets['end_target_year'].fillna(0).astype(int)
all_targets['end_target_year'] = all_targets['end_target_year'].replace(0,'No target')
all_targets['end_target_year'] = all_targets['end_target_year'].astype(str)

In [4]:
all_targets.drop(all_targets[all_targets['test_dataset'] == 'x'].index, inplace=True)
all_targets.drop(all_targets[all_targets['val_dataset'] == 'x'].index, inplace=True)

In [5]:
all_targets.value_counts('end_target')

end_target
Carbon neutral(ity)           114
Net zero                      103
No target                     102
Emissions reduction target    100
Name: count, dtype: int64

In [6]:
percentages = all_targets['end_target'].value_counts(normalize=True) * 100
print(percentages)

end_target
Carbon neutral(ity)           27.207637
Net zero                      24.582339
No target                     24.343675
Emissions reduction target    23.866348
Name: proportion, dtype: float64


In [ ]:
from datasets import Dataset
hf_dataset = Dataset.from_pandas(all_targets)

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

upload_key = os.environ['HUGGINGFACE_UPLOAD']

repo_name = 'chris7374/esg-net-zero'
revision = '100_per_class_v3'
hf_dataset.push_to_hub(repo_name, revision=revision, token=upload_key)